Univariate data analyses - NHANES case study - https://www.cdc.gov/nchs/nhanes/index.htm

NHANES is a U.S. Government study with data properly anomonized to protect privacy.  The study captures body biometrics along
with blood pressure and education levels.  The study results for the 2015-2016 year of the study are used in this notebook. 
Here is a link for the data file:  https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2015

This notebook will compare and contrast using numpy and the Arkouda equivalents to perform some exploratory descriptive statistical analysis of the dataset.


In [ ]:
#
# setup pandas and other std packages
#
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
#
# setup arkouda packages and connection to arkouda
#

import arkouda as ak
import pandas as pd
import numpy as np

ak.connect()


In [ ]:
#
# load NHANES data from file into Pandas and dump some results
#

pd_df = pd.read_csv("NHanes/nhanes_2015_2016.csv")
pd_df.head()


In [ ]:
#
# LOAD NHANES data from pandas into arkouda dataframe and dump some results
#
ak_df = ak.DataFrame(pd_df)
ak_df.head()

The numbers 1, 2, 3, 4, 5, 9 seen below are integer codes for the 6 possible non-missing values of the DMDEDUC2 variable. The meaning of these codes is given in the NHANES codebook. This table shows, for example, that 1621 people in the data file have DMDEDUC=4, which indicates that the person has completed some college, but has not graduated with a four-year degree.

In [ ]:
pd_df.DMDEDUC2.value_counts()

In [ ]:
#
# We need to force these categorical values, which are floats, to int values
#
result = ak.value_counts(ak.cast(ak_df["DMDEDUC2"],"int"))
display(result)

Notice that the Arkouda count method includes missing values that show up as a category of zero(Pandas by contrast does not show these values).  Below you will see the results from a Pandas sum and an arkouda array sum which has difference of 261 (the number of missing values, aka NaN)

In [ ]:
#
# Pandas sum of unique column values
#
pd_df.DMDEDUC2.value_counts().sum()

In [ ]:
#
# Arkouda sum of unique colum values, second array contains sum the first contast the categorical values
#
result[1].sum()

In [ ]:
#
# quick way to determine the missing values in the Pandas dataframe (aka NaN)
#
pd_df.DMDEDUC2.isnull().sum() 

In [ ]:
#
# equivalent quick way to determine the missing values in the Arkouda dataframe (aka NaN)
#
ak.isnan(ak_df["DMDEDUC2"]).sum()

For many purposes it is more relevant to consider the proportion of the sample with each of the possible category values, rather than the number of people in each category. 

In [ ]:
#
# convert value counts to a proportion
#
p = pd_df.DMDEDUC2.value_counts()  # convert value counts to a proportion
p/p.sum()

In [ ]:
#
# We need to work a bit harder to get the porportion values in Arkouda
#
display("portions for column DMDEDUC2")
display(result[0][1:])  # need to ignore zero counts as they are for nan columns
display(result[1][1:]/result[1][1:].sum())

A quick way to get a set of numerical summaries for a quantitative variable is with the describe data frame method. Below we demonstrate how to do this using the body weight variable (BMXWT). As with many surveys, some data values are missing, so we explicitly drop the missing cases using the dropna method before generating the summaries.

In [ ]:
#
# Pandas describe to get descriptive stats on a column
#
pd_df.BMXWT.dropna().describe()

In [ ]:
#
# Arkouda to get descriptive stats on a column
# We need to drop nan values and add median and percentile using numpy, since Arkouda does not have implementations for these
# functions.
#
ak_bmxwt = ak_df[~ak.isnan(ak_df["BMXWT"])]["BMXWT"] # get values that are not nan for "BMXWT" column

display(np.count_nonzero(ak_bmxwt.to_ndarray()))
display(ak.mean(ak_bmxwt))
display(np.median(ak_bmxwt.to_ndarray()))
display(ak.std(ak_bmxwt))
display(ak.min(ak_bmxwt))
display(np.percentile(ak_bmxwt.to_ndarray(),25))
display(np.percentile(ak_bmxwt.to_ndarray(),50))
display(np.percentile(ak_bmxwt.to_ndarray(),75))
display(ak.max(ak_bmxwt))


The blood pressure measurements are captured in the fields with the "BPX" prefix.  "BPXSYS1 & 2" are the systolic blood pressure values
and "BPXDI1 & 2" are the diastolic blood pressure values.

A person is generally considered to have pre-hypertension when their systolic blood pressure is between 120 and 139, or their diastolic blood pressure is between 80 and 89. Considering only the systolic condition, we can calculate the proprotion of the NHANES sample who would be considered to have pre-hypertension.

In [ ]:
#
# First we will calculate the mean values in numpy
#

In [ ]:
display(np.mean((pd_df.BPXSY1 >= 120) & (pd_df.BPXSY2 <= 139)))

display(np.mean((pd_df.BPXDI1 >= 80) & (pd_df.BPXDI2 <= 89)))

In [ ]:
#
# Now we will calculate the same values in Arkouda
# Again we will need to force a cast of the float values to int values in order to call Arkouda's
# stat functions
#

display(ak.mean((ak.cast(ak_df["BPXSY1"],"int") >= 120) & (ak.cast(ak_df["BPXSY2"],"int") <= 139)))

display(ak.mean((ak.cast(ak_df["BPXDI1"],"int") >= 80) & (ak.cast(ak_df["BPXDI2"],"int") <= 89)))


Finally we calculate the proportion of NHANES subjects who are pre-hypertensive based on either systolic or diastolic blood pressure. Since some people are pre-hypertensive under both criteria, the proportion below is less than the sum of the two proportions calculated above.

Since the combined systolic and diastolic condition for pre-hypertension is somewhat complex, below we construct temporary variables 'a' and 'b' that hold the systolic and diastolic pre-hypertensive status separately, then combine them with a "logical or" to obtain the final status for each subject.

In [ ]:
#
# pandas calulations
#
a = (pd_df.BPXSY1 >= 120) & (pd_df.BPXSY2 <= 139)
b = (pd_df.BPXDI1 >= 80) & (pd_df.BPXDI2 <= 89)
display(np.mean(a | b))  

In [ ]:
#
# arkoudas dataframe calcs
#
a = (ak.cast(ak_df["BPXSY1"],"int") >= 120) & (ak.cast(ak_df["BPXSY2"],"int") <= 139)
b = (ak.cast(ak_df["BPXDI1"],"int") >= 80) & (ak.cast(ak_df["BPXDI2"],"int") <= 89)

display(ak.mean(a | b))  

Blood pressure measurements are affected by a phenomenon called "white coat anxiety", in which a subject's bood pressure may be slightly elevated if they are nervous when interacting with health care providers. Typically this effect subsides if the blood pressure is measured several times in sequence. In NHANES, both systolic and diastolic blood pressure are meausred three times for each subject (e.g. BPXSY2 is the second measurement of systolic blood pressure). We can calculate the extent to which white coat anxiety is present in the NHANES data by looking at the mean difference between the first two systolic or diastolic blood pressure measurements.

In [ ]:
#
# pandas/numpy calulations (systolic and diastolic)
# pandas has smoke and mirrors to handle NaN
#
display(np.mean(pd_df.BPXSY1 - pd_df.BPXSY2))
display(np.mean(pd_df.BPXDI1 - pd_df.BPXDI2))

In [ ]:
#
# arkouda calulations (systolic and diastolic)
# Need a mask to handle NaN columns
# Note the mask technique may increase the precision of the arkouda calculated values in prior cells
#
from functools import reduce
mask = reduce(lambda x, y: x&y,[~ak.isnan(ak_df[c_name]) if ak_df[c_name].dtype == ak.float64 else ak.ones(ak_df.size, 'bool') for c_name in ["BPXSY1", "BPXSY2"]])
masked_df = ak_df[mask]
display(ak.mean(masked_df["BPXSY1"] - masked_df["BPXSY2"]))
display(ak.mean(masked_df["BPXDI1"] - masked_df["BPXDI2"]))


Graphical summaries
Quantitative variables can be effectively summarized graphically. Below we see the distribution of body weight (in Kg), shown as a histogram. It is evidently right-skewed.

In [ ]:
#
# pandas body weight plot 
#

sns.histplot(
    pd_df.BMXWT.dropna(), kde=True,
    stat="density", kde_kws=dict(cut=3))


In [ ]:
#
# arkouda body weight plot
# Note: I had difficulty getting seaborn to accept an arkouda array, so I converted the arkouda array to 
# an ndarray.
#
masked_df = ak_df[mask]
ak_bmxwt = masked_df["BMXWT"]

sns.histplot(
    ak_bmxwt.to_ndarray(), kde=True,
    stat="density", kde_kws=dict(cut=3))

Next we look at the histogram of systolic blood pressure measurements. You can see that there is a tendency for the measurements to be rounded to the nearest 5 or 10 units.

In [ ]:
#
# pandas plot of systolic pressure
#
sns.histplot(
    pd_df.BPXSY1.dropna(), kde=True,
    stat="density", kde_kws=dict(cut=3))


In [ ]:
#
# arkouda plot of systolic pressure, again ndarray is the currency of the realm for plotting
#
masked_df = ak_df[mask]
ak_bpxsy1 = masked_df["BPXSY1"]

sns.histplot(
    ak_bpxsy1.to_ndarray(), kde=True,
    stat="density", kde_kws=dict(cut=3))

To compare several distributions, we can use side-by-side boxplots. Below we compare the distributions of the first and second systolic blood pressure measurements (BPXSY1, BPXSY2), and the first and second diastolic blood pressure measurements (BPXDI1, BPXDI2). As expected, diastolic measurements are substantially lower than systolic measurements. Above we saw that the second blood pressure reading on a subject tended on average to be slightly lower than the first measurement. This difference was less than 1 mm/Hg, so is not visible in the "marginal" distributions shown below.

In [ ]:
#
# pandas boxplot of systolic diastolic pressures
#
bp = sns.boxplot(data=pd_df.loc[:, ["BPXSY1", "BPXSY2", "BPXDI1", "BPXDI2"]])
_ = bp.set_ylabel("Blood pressure in mm/Hg")

In [ ]:
#
# arkouda boxplot of systolic diastolic pressures, again converting the arkouda dataframe back
# to a Pandas dataframe makes the plotting task easier
#
masked_df = ak_df[mask]["BPXSY1", "BPXSY2", "BPXDI1", "BPXDI2"]
df_x = masked_df.to_pandas()

bp = sns.boxplot(data=df_x.loc[:,["BPXSY1", "BPXSY2", "BPXDI1", "BPXDI2"]])
_ = bp.set_ylabel("Blood pressure in mm/Hg")

In [ ]:
#
# tear down connection to arkouda
#
ak.disconnect()
#ak.shutdown()